In [1]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

In [2]:
user=pd.read_csv('data.csv')

In [3]:
user.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 15463110 entries, 0 to 15463109
Data columns (total 6 columns):
 #   Column         Dtype 
---  ------         ----- 
 0   user_id        int64 
 1   item_id        int64 
 2   behavior_type  int64 
 3   user_geohash   object
 4   item_category  int64 
 5   time           object
dtypes: int64(4), object(2)
memory usage: 707.8+ MB


In [4]:
# 快速查看统计信息
user.describe()

,user_id,item_id,behavior_type,item_category
count,1.546311e+07,1.546311e+07,1.546311e+07,1.546311e+07
mean,7.015207e+07,2.023169e+08,1.153780e+00,6.827181e+03
std,4.572019e+07,1.167524e+08,5.440445e-01,3.810410e+03
min,4.920000e+02,3.700000e+01,1.000000e+00,2.000000e+00
25%,3.021406e+07,1.014015e+08,1.000000e+00,3.687000e+03
50%,5.638708e+07,2.022669e+08,1.000000e+00,6.054000e+03
75%,1.166482e+08,3.035247e+08,1.000000e+00,1.025800e+04
max,1.424430e+08,4.045625e+08,4.000000e+00,1.408000e+04


In [5]:
# 删除重复值
user.drop_duplicates(keep='last',inplace=True)

In [6]:
# 将time转换为datetime格式
user['time']=pd.to_datetime(user['time'])

In [7]:
# 提取出日期和时间
user['dates'] = user.time.dt.date
user['month'] = user.dates.values.astype('datetime64[M]')
user['hours'] = user.time.dt.hour

In [8]:
# 转换数据类型
user['behavior_type']=user['behavior_type'].apply(str)
user['user_id']=user['user_id'].apply(str)
user['item_id']=user['item_id'].apply(str)

In [9]:
# 统计每日PV和UV数据
pv_day=user[user.behavior_type=="1"].groupby("dates")["behavior_type"].count()
uv_day=user[user.behavior_type=="1"].drop_duplicates(["user_id","dates"]).groupby("dates")["user_id"].count()

In [10]:
# 分析每天的pv与uv的趋势
# 加载库
import pyecharts.options as opts
from pyecharts.charts import Line
from pyecharts.charts import Grid
import numpy as np

# 做出每天的pv与uv趋势图
attr=list(pv_day.index)
pv=(
    Line(init_opts=opts.InitOpts(width="1000px",height="500px"))
    .add_xaxis(xaxis_data=attr)
    .add_yaxis(
        "pv",
        np.around(pv_day.values/10000,decimals=2),
        label_opts=opts.LabelOpts(is_show=False)
    )
    .add_yaxis(
        series_name="uv",
        yaxis_index=1,
        y_axis=np.around(uv_day.values/10000,decimals=2),
        label_opts=opts.LabelOpts(is_show=False),
    )
    .extend_axis(
        yaxis=opts.AxisOpts(
            name="uv",
            type_="value",
            min_=0,
            max_=1.6,
            interval=0.4,
            axislabel_opts=opts.LabelOpts(formatter="{value} 万人"),
        )
    )
    .set_global_opts(
        tooltip_opts=opts.TooltipOpts(
            is_show=True,trigger="axis",axis_pointer_type="cross"
        ),
        xaxis_opts=opts.AxisOpts(
            type_="category",
            axispointer_opts=opts.AxisPointerOpts(is_show=True,type_="shadow"),
        ),
        yaxis_opts=opts.AxisOpts(
            name="pv",
            type_="value",
            min_=0,
            max_=100,
            interval=20,
            axislabel_opts=opts.LabelOpts(formatter="{value} 万次"),
            axistick_opts=opts.AxisTickOpts(is_show=True),
            splitline_opts=opts.SplitLineOpts(is_show=True),
        ),
        title_opts=opts.TitleOpts(title="pv与uv趋势图"),
    )
)

pv.render_notebook()

In [12]:
# pv、uv差异分析（by day）
pv_uv = pd.concat([pv_day, uv_day], join='outer', axis=1)
pv_uv.columns = ['pv_day', 'uv_day']

new_day=pv_uv.diff()
new_day.columns=['new_pv','new_uv']
new_day

,new_pv,new_uv
dates,,
2023-11-18,NaN,NaN
2023-11-19,9121.0,61.0
2023-11-20,-12504.0,-71.0
2023-11-21,-20029.0,-228.0
2023-11-22,18068.0,-122.0
2023-11-23,37567.0,432.0
2023-11-24,-7484.0,140.0
2023-11-25,-17090.0,-178.0
2023-11-26,-6583.0,-138.0


In [13]:
attr = new_day.index
v = new_day.new_uv
w = new_day.new_pv

li=(
    Line(init_opts=opts.InitOpts(width="1000px",height="500px"))
    .add_xaxis(xaxis_data=attr)
    .add_yaxis(
        "新增pv",
        w,
        label_opts=opts.LabelOpts(is_show=False)
    )
    .extend_axis(
        yaxis=opts.AxisOpts(
            name="新增uv",
            type_="value",
            min_=-2000,
            max_=1600,
            interval=400,
            axislabel_opts=opts.LabelOpts(formatter="{value}"),
        )
    )
     .set_global_opts(
        tooltip_opts=opts.TooltipOpts(
            is_show=True, trigger="axis", axis_pointer_type="cross"
        ),
        xaxis_opts=opts.AxisOpts(
            type_="category",
            axispointer_opts=opts.AxisPointerOpts(is_show=True, type_="shadow"),
        ),
        yaxis_opts=opts.AxisOpts(
            name="新增pv",
            type_="value",
            min_=-350000,
            max_=250000,
            interval=100000,
            axislabel_opts=opts.LabelOpts(formatter="{value}"),
            axistick_opts=opts.AxisTickOpts(is_show=True),
            splitline_opts=opts.SplitLineOpts(is_show=True),
        ),
         title_opts=opts.TitleOpts(title="pv、uv差异分析"),
    )
)
il=(
    Line()
    .add_xaxis(xaxis_data=attr)
    .add_yaxis("新增uv",v,yaxis_index='1',label_opts=opts.LabelOpts(is_show=False),)
)
c=li.overlap(il)
c.render_notebook()

In [14]:
# 不同时期用户行为分析
shopping_cart= user[user.behavior_type == '3'].groupby('dates')['behavior_type'].count()
collect=user[user.behavior_type=='2'].groupby('dates')['behavior_type'].count()
buy=user[user.behavior_type=='4'].groupby('dates')['behavior_type'].count()

attr_a=list(shopping_cart.index)
v_1=shopping_cart.values.tolist()
v_2=collect.values.tolist()
v_3=buy.values.tolist()

b=(
    Line()
    .add_xaxis(xaxis_data=attr_a)
    .add_yaxis(
        "加购人数",
        v_1,
        label_opts=opts.LabelOpts(is_show=False)
    )
    .add_yaxis(
        "收藏人数",
        v_2,
        label_opts=opts.LabelOpts(is_show=False)
    )
    .add_yaxis(
        "购买人数",
        v_3,
        label_opts=opts.LabelOpts(is_show=False)
    )
    .set_global_opts(title_opts=opts.TitleOpts(title="不同时期用户行为数据"))
)
b.render_notebook()

In [15]:
# 把dates列转换为datetime类型
user['dates']=pd.to_datetime(user['dates'])

In [16]:
# 选取活动数据子集和日常数据子集
active=user[user["dates"].isin(["2023/12/11","2023/12/12","2023/12/13"])]
daily=user[~user["dates"].isin(["2023/12/11","2023/12/12","2023/12/13"])]

In [17]:
# 活动期间不同时段的用户行为分析
from pyecharts.charts import Bar

# 活动数据
cart_h= active[active.behavior_type == '3'].groupby('hours')['behavior_type'].count()
collect_h=active[active.behavior_type=='2'].groupby('hours')['behavior_type'].count()
buy_h=active[active.behavior_type=='4'].groupby('hours')['behavior_type'].count()
uv_h=active[active.behavior_type== '1'].groupby('hours')['user_id'].count()

attr_h=list(cart_h.index)
h1=np.around(cart_h.values/3,decimals=0).tolist()
h2=np.around(collect_h.values/3,decimals=0).tolist()
h3=np.around(buy_h.values/3,decimals=0).tolist()
h4=np.around(uv_h.values/3,decimals=0).tolist()


h=(
    Line(init_opts=opts.InitOpts(width="1000px",height="500px"))
    .add_xaxis(xaxis_data=attr_h)
    .add_yaxis(
        "加购人数",
        h1,
        label_opts=opts.LabelOpts(is_show=False)
    )
    .add_yaxis(
        "收藏人数",
        h2,
        label_opts=opts.LabelOpts(is_show=False)
    )
    .add_yaxis(
        "购买人数",
        h3,
        label_opts=opts.LabelOpts(is_show=False)
    )
    .set_global_opts(
        xaxis_opts=opts.AxisOpts(axislabel_opts=opts.LabelOpts(rotate=15)),
        title_opts=opts.TitleOpts(title="日均各时段活动用户行为",pos_top="48%"),
        legend_opts=opts.LegendOpts(pos_top="48%"),
    )
)
bar=(
    Bar()
    .add_xaxis(xaxis_data=attr_h)
    .add_yaxis(
    "浏览人数",
        h4,
        label_opts=opts.LabelOpts(is_show=False)
    )
    .set_global_opts(
        title_opts=opts.TitleOpts(title="活动uv对比数据"),
    )
)

ggrid = (
    Grid()
    .add(bar, grid_opts=opts.GridOpts(pos_bottom="60%"))
    .add(h, grid_opts=opts.GridOpts(pos_top="60%"))
)
ggrid.render_notebook()

In [24]:
# 日常期间不同时段的用户行为分析
# 日常数据
cart_d= daily[daily.behavior_type == '3'].groupby('hours')['behavior_type'].count()
collect_d=daily[daily.behavior_type=='2'].groupby('hours')['behavior_type'].count()
buy_d=daily[daily.behavior_type=='4'].groupby('hours')['behavior_type'].count()
uv_d=daily[daily.behavior_type== '1'].groupby('hours')['user_id'].count()

attr_d=list(cart_d.index)
d1=np.around(cart_d.values/28,decimals=0).tolist()
d2=np.around(collect_d.values/28,decimals=0).tolist()
d3=np.around(buy_d.values/28,decimals=0).tolist()
d4=np.around(uv_d.values/28,decimals=0).tolist()

d=(
    Line(init_opts=opts.InitOpts(width="1000px",height="500px"))
    .add_xaxis(xaxis_data=attr_d)
    .add_yaxis(
        "加购人数",
        d1,
        label_opts=opts.LabelOpts(is_show=False)
    )
    .add_yaxis(
        "收藏人数",
        d2,
        label_opts=opts.LabelOpts(is_show=False)
    )
    .add_yaxis(
        "购买人数",
        d3,
        label_opts=opts.LabelOpts(is_show=False)
    )
    .set_global_opts(
        xaxis_opts=opts.AxisOpts(axislabel_opts=opts.LabelOpts(rotate=15)),
        title_opts=opts.TitleOpts(title="日均各时段活动用户行为",pos_top="48%"),
        legend_opts=opts.LegendOpts(pos_top="48%"),
    )
)
y=(
    Bar()
    .add_xaxis(xaxis_data=attr_d)
    .add_yaxis(
    "浏览人数",
        d4,
        label_opts=opts.LabelOpts(is_show=False)
    )
    .set_global_opts(
        title_opts=opts.TitleOpts(title="日常uv对比数据"),
    )
)

gggrid = (
    Grid()
    .add(y, grid_opts=opts.GridOpts(pos_bottom="60%"))
    .add(d, grid_opts=opts.GridOpts(pos_top="60%"))
)
gggrid.render_notebook()

In [19]:
# 活动时购买率
hour_buy_user_num = active[active.behavior_type == '4'].drop_duplicates(['user_id','dates', 'hours']).groupby('hours')['user_id'].count()
hour_active_user_num = active.drop_duplicates(['user_id','dates', 'hours']).groupby('hours')['user_id'].count()
hour_buy_rate = hour_buy_user_num / hour_active_user_num
attr_o = list(hour_buy_user_num.index)
vo_2 =np.around(hour_buy_rate.values,decimals=2) 
# 日常时购买率
hour_buy_daily_num = daily[daily.behavior_type == '4'].drop_duplicates(['user_id','dates', 'hours']).groupby('hours')['user_id'].count()
hour_active_daily_num = daily.drop_duplicates(['user_id','dates', 'hours']).groupby('hours')['user_id'].count()
daily_buy_rate = hour_buy_daily_num / hour_active_daily_num
vi_2 =np.around(daily_buy_rate.values,decimals=2) 


hbu=(
    Line()
    .add_xaxis(xaxis_data=attr_o)
    .add_yaxis(
        "日常购买率",
        vi_2,
    )
    .add_yaxis(
        "活动购买率",
        vo_2,
    )
    .set_global_opts(title_opts=opts.TitleOpts(title="不同时段购买率"))
)

hbu.render_notebook()

In [20]:
# 活动期间的转化漏斗
from pyecharts.charts import Funnel

# 活动转化
a_pv=active[active.behavior_type=="1"]["user_id"].count()
a_cart=active[active.behavior_type=="3"]["user_id"].count()
a_collect=active[active.behavior_type=="2"]["user_id"].count()
a_buy=active[active.behavior_type=="4"]["user_id"].count()

a_attr=["点击","加入购物车","收藏","购买"]
values=[np.around((a_pv/a_pv*100),2),
        np.around((a_cart/a_pv*100),2),
        np.around((a_collect/a_pv*100),2),
        np.around((a_buy/a_pv*100),2),
       ]

data = [[a_attr[i], values[i]] for i in range(len(a_attr))]

a=(
    Funnel()
    .add(
        series_name="用户行为",
        data_pair=data,
        gap=2,
        tooltip_opts=opts.TooltipOpts(trigger="item", formatter="{a} <br/>{b} : {c}%",is_show=True),
        label_opts=opts.LabelOpts(is_show=True, position="ourside"),
        itemstyle_opts=opts.ItemStyleOpts(border_color="#fff", border_width=1),
    )
    .set_global_opts(title_opts=opts.TitleOpts(title="用户转化漏斗", subtitle="活动"))
)

a.render_notebook()

In [21]:
# 日常期间的转化漏斗
# 日常转化
l_pv=daily[daily.behavior_type=="1"]["user_id"].count()
l_cart=daily[daily.behavior_type=="3"]["user_id"].count()
l_collect=daily[daily.behavior_type=="2"]["user_id"].count()
l_buy=daily[daily.behavior_type=="4"]["user_id"].count()

l_attr=["点击","加入购物车","收藏","购买"]
valuel=[np.around((l_pv/l_pv*100),2),
        np.around((l_cart/l_pv*100),2),
        np.around((l_collect/l_pv*100),2),
        np.around((l_buy/l_pv*100),2),
       ]

datal = [[l_attr[i], valuel[i]] for i in range(len(l_attr))]

dy=(
    Funnel()
    .add(
        series_name="用户行为",
        data_pair=datal,
        gap=2,
        tooltip_opts=opts.TooltipOpts(trigger="item", formatter="{a} <br/>{b} : {c}%",is_show=True),
        label_opts=opts.LabelOpts(is_show=True, position="ourside"),
        itemstyle_opts=opts.ItemStyleOpts(border_color="#fff", border_width=1),
    )
    .set_global_opts(title_opts=opts.TitleOpts(title="用户转化漏斗", subtitle="日常"))
)
dy.render_notebook()

In [31]:
# 使用Page类将多个图表组合起来
from pyecharts.charts import Page
page.add(pv, c, b, ggrid, gggrid, hbu, a, dy)

# 保存为HTML文件
page.render("multi_charts.html")

'C:\\Users\\23924\\Desktop\\数据分析师课程\\Python\\Python数分项目\\电商用户行为分析\\multi_charts.html'